# **Task 1: Parsing Raw Files**
Step 1: Achieve txt files parsing to csv format

Step 2: Achieve xlsx file parsing to csv format

Step 3: Combine all data together and output task1.csv

Step 4: Achieve txt files parsing to json format

Step 5: Achieve xlsx file parsing to json format

Step 6: Combine all data together and output task1_output.json

Step 7: Generate necessary files and report.

In [1]:
# ignore FutureWarning, UserWarning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [3]:
import re
import os
import json
import pandas as pd
from datetime import datetime, timezone

**Step 1:** Achieve txt files parsing to csv format

List all files in a directory, filtering the files using a regular expression. This line filters the file list to include only files that match the pattern group123_<number>.txt (for example, group123_1.txt, group123_2.txt). Regular expressions ensure that only these archives are processed.

In [4]:
# input txt files
file_dir = os.listdir('/Original_data')
files = [f for f in file_dir if re.match(r'group123_\d+\.txt', f)]

creat an empty list, which will hold DataFrames created from the data extracted from the text files.

In [5]:
df_list = []

1. Loop through each filtered text file, open and read the contents of each file, and store it in the data variable. Retrieve records using regular expressions, "record(.*?)record" code searches for all text contained within the tag. The re.DOTALL flag allows matching newlines, enabling the capture of multiple lines of records.

2. Create three empty lists to store gmap_id, text, and resp values extracted from each record.

3. Extract fields from each record, Within each record, the code uses regular expressions to search for and extract gmap_id, text, and resp values. These values are extracted from tags like gmap_id, review, or response.

4. Append extracted data to lists, the extracted values are stripped of any leading or trailing space and then added to their respective lists.

5. After processing all records in a file, create a DataFrame from the gmap_id, text, and resp lists. Then, append the DataFrame to the list.

In [6]:
for f_name in files:
  f_path = os.path.join('/Original_data', f_name)

  with open(f_path, 'r', encoding='utf-8') as file:
      data = file.read()

  records = re.findall(r'<record>(.*?)</record>', data, re.DOTALL)

  gmap_list = []
  text_list = []
  resp_list = []

  for record in records:
    gmap_id = re.search(r'<\s*[gG]map[_IDid]*\s*>\s*([^<]+)\s*<', record)
    text = re.search(r'<\s*(?:[rR]eview|[tT]ext)\s*>\s*([^<]+)\s*<', record)
    resp = re.search(r'<\s*(?:[rR]esp(?:onse)?)\s*>\s*([^<]+)\s*<', record)

    gmap_list.append(gmap_id.group(1).strip())
    text_list.append(text.group(1).strip())
    resp_list.append(resp.group(1).strip())

  df = pd.DataFrame({
      'gmap_id': gmap_list,
      'text': text_list,
      'resp': resp_list
  })
  df_list.append(df)

**Step 2:** Achieve xlsx file parsing to csv format

In [7]:
xls = pd.ExcelFile('/Original_data/group123.xlsx')

1. Reads the current sheet into a DataFrame named sheet_df, then, removes any rows in the DataFrame where the gmap_id column is NaN. This ensures that only rows with valid gmap_id values are processed.
2. converting text&resp to a string and replacing 'nan' with 'None'.
3. This line extracts the gmap_id, text, and resp columns from the cleaned DataFrame and appends them to the df_list.
4. Combines all the DataFrames stored in df_list into a single DataFrame called df_concat.

In [8]:
for sheet_name in xls.sheet_names:
  sheet_df = pd.read_excel(xls, sheet_name=sheet_name)
  sheet_df = sheet_df.dropna(subset = ['gmap_id'])
  sheet_df['text'] = sheet_df['text'].astype(str)
  sheet_df['text'] = sheet_df['text'].replace('nan', 'None')
  sheet_df['resp'] = sheet_df['resp'].astype(str)
  sheet_df['resp'] = sheet_df['resp'].replace('nan', 'None')
  df_list.append(sheet_df[['gmap_id', 'text', 'resp']])

<ipython-input-8-093b650650c0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sheet_df['text'] = sheet_df['text'].astype(str)
<ipython-input-8-093b650650c0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sheet_df['text'] = sheet_df['text'].replace('nan', 'None')
<ipython-input-8-093b650650c0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

1. Groups the combined DataFrame by the gmap_id column. This means that all rows with the same gmap_id are grouped together.

2. Aggregates the grouped data with calculations:
  A. number of reviews (i.e., the number of rows in each group).
  B. number of non-'None' texts in each group.
  C. number of non-'None' responses in each group.
3. Converts the grouped data back into a DataFrame format with a new index.

In [9]:
df_concat = pd.concat(df_list, ignore_index=True)

df_grouped = df_concat.groupby('gmap_id').agg(
    review_count=('text', 'size'),
    review_text_count=('text', lambda x: (x.str.lower() != 'none').sum()),
    response_count=('resp', lambda x: (x.str.lower() != 'none').sum())
).reset_index()

**Step 3:** Combine all data together and output task1_123.csv

In [10]:
df_grouped.to_csv('/task1.csv', index=False)

# txt files and xlsx file to JSON
Step 4: Achieve txt files parsing to json format

Step 5: Achieve xlsx file parsing to json format

Step 6: Combine all data together and output task1_output.json

Step 7: Generate necessary files and report.

Create functions

In [11]:
# Function to transfer Unix timestamp to UTC
def transfer_date_to_utc(review_time):
  return datetime.fromtimestamp(int(review_time) / 1000, timezone.utc).strftime('%Y-%m-%d %H:%M:%S')

  This function cleans a given review by:
1. Extracting the English part of the review if it's translated by Google.
2. Converting the text to lowercase.
3. Removing emojis from the text.
4. Ensuring that an empty or irrelevant review is replaced with "None".




In [12]:
def review_cleaning(review):
  # extract english reviews
  english_review_match = re.search(r"\(Translated by Google\)\s*(.*?)\s*\(Original\)", review)
  english_review = english_review_match.group(1).strip() if english_review_match else review
  # ensure text is in a lowercase
  lowercase_review = english_review.lower()
  emoji_pattern = re.compile(
      "["
      "\U0001F600-\U0001F64F"
      "\U0001F300-\U0001F5FF"
      "\U0001F680-\U0001F6FF"
      "\U0001F1E0-\U0001F1FF"
      "\U00002702-\U000027B0"
      "\U000024C2-\U0001F251"
      "]+", flags=re.UNICODE
  )
  # remove emojis
  lowercase_review = emoji_pattern.sub(r'', lowercase_review).strip()

  # There are 2 situations. 1. after removing emojis, review can be empty, we add "None". 2. At the beginning it shows "None"
  return lowercase_review if lowercase_review and lowercase_review != "none" else "None"

This function extracts the dimensions (width and height) from picture URLs provided in a text string and returns them as a list of lists.

In [13]:
# Function to get dimensions from picture URLs in txt files
def get_dimensions(pics):
  dim_list = []
  urls = re.findall(r'https?://[^\'\]}]+', pics)
  for url in urls:
      dim_match = re.search(r'w(\d+)-h(\d+)', url)
      if dim_match:
          width, height = dim_match.groups()
          dim_list.append([height, width])
  return dim_list

This function cleans the review text, converts the review time to UTC, and extracts relevant information into a structured format. Then, return dictionary containing processed review data with keys for user ID, review time, review rating, cleaned review text, presence of pictures, picture dimensions, and presence of a response.

Processing txt files by these steps:
1. Extract all records from the file
2. Process each record individually
3. Extract the user ID from the record
4. Convert the review time to UTC
5. Extract the review rating and convert it to a float
6. Extract and clean the review text
7. Extract picture URLs and determine if pictures are present
8. Check if there is a response to the review
9. Create a dictionary to store the processed review data
10. Append the processed review to the list of reviews for the gmap_id
11. Update the earliest & latest review date if necessary

In [16]:
# Processing txt files
txt_files = [f for f in file_dir if f.endswith('.txt')]
out_dic = {}

for f_name in txt_files:
  f_path = os.path.join('/Original_data', f_name)
  with open(f_path, 'r', encoding='utf-8') as file:
      data = file.read()

  records = re.findall(r'<record>(.*?)</record>', data, re.DOTALL)

  # Extract the gmap_id from the record
  for record in records:
      gmap_id = re.search(r'<\s*[gG]map[_IDid]*\s*>\s*([^<]+)\s*<', record).group(1).strip()

      if gmap_id not in out_dic:
          out_dic[gmap_id] = {
              "reviews": [],
              "earliest_review_date": None,
              "latest_review_date": None
          }

      user_id_match = re.search(r'<\s*[uU]ser[_.IDid]*\s*>\s*([^<]+)\s*<', record)
      user_id = user_id_match.group(1).strip() if user_id_match else "None"

      review_time_utc = transfer_date_to_utc(
          re.search(r'<\s*(?:[tT]ime|[dD]ate)\s*>\s*([^<]+)\s*<', record).group(1)).strip()

      rating_match = re.search(r'<\s*(?:[rR]at(?:e|ing))\s*>\s*([^<]+)\s*<', record).group(1)
      # If conversion fails, keep the original value
      try:
          rating = float(rating_match)
      except ValueError:
          rating = rating_match

      review_text_match = re.search(r'<\s*(?:[rR]eview|[tT]ext)\s*>\s*([^<]+)\s*<', record).group(1).strip()
      # if the original content is empty <><> or no review tags -> None
      review_text = review_cleaning(review_text_match) if review_text_match else "None"

      pics_match = re.search(r'<\s*(?:[pP]ic(?:ture)?s)\s*>\s*([^<]+)\s*<', record).group(1).strip()
      pics = pics_match if pics_match else "None"

      # Mark "Y" if pictures are present, otherwise "N"
      if_pic = "Y" if pics.lower() != "none" else "N"

      # Extract picture dimensions if pictures are present
      pic_dim = get_dimensions(pics) if if_pic == 'Y' else '[]'

      response_match = re.search(r'<\s*(?:[rR]esp(?:onse)?)\s*>\s*([^<]+)\s*<', record)
      if_response = "Y" if response_match and response_match.group(1).lower() != "none" else "N"

      review_inner = {
          "user_id": user_id,
          "time": review_time_utc,
          "review_rating": rating,
          "review_text": review_text,
          "if_pic": if_pic,
          "pic_dim": pic_dim,
          "if_response": if_response
      }

      out_dic[gmap_id]["reviews"].append(review_inner)

      # Update earliest review dates
      if not out_dic[gmap_id]["earliest_review_date"] or review_time_utc < out_dic[gmap_id]["earliest_review_date"]:
          out_dic[gmap_id]["earliest_review_date"] = review_time_utc
      # Update latest review dates
      if not out_dic[gmap_id]["latest_review_date"] or review_time_utc > out_dic[gmap_id]["latest_review_date"]:
          out_dic[gmap_id]["latest_review_date"] = review_time_utc

Processing xlsx file by these steps:
1. Process each sheet in the Excel file
2. Combine data from all sheets into a single DataFrame
3. Process the combined data by grouping it by gmap_id
4. Update the out_dic dictionary with the processed data

In [14]:
# Function to process reviews from xlsx files
def process_review(row):
  clean_text = review_cleaning(row['text']) if pd.notna(row['text']) else "None"
  return {
      "user_id": row['user_id'],
      "time": transfer_date_to_utc(row['time']),
      "review_rating": float(row['rating']),
      "review_text": clean_text,
      "If_pic": "Y" if pd.notna(row['pics']) else "N",
      "pic_dim": [],
      "If_response": "Y" if pd.notna(row['resp']) else "N"
  }

In [17]:
# Processing xlsx files
xlsx_files = [f for f in file_dir if f.endswith('.xlsx')]

for f_name in xlsx_files:
  f_path = os.path.join('/Original_data', f_name)
  xls = pd.ExcelFile(f_path)
  sheet_names = xls.sheet_names
  xls_data = []

  for sheet_name in sheet_names:
      sheet_data = pd.read_excel(xls, sheet_name=sheet_name)
      valid_data = sheet_data.dropna(subset=['gmap_id'])
      xls_data.append(valid_data)

  # all valid data into one dataframe
  combined_data = pd.concat(xls_data, ignore_index=True)

  # Apply the process_review function to each row in the group and convert the result to a list
  for gmap_id, group in combined_data.groupby('gmap_id'):
      reviews = group.apply(process_review, axis=1).tolist()

      #convert to UTC
      earliest_review_date = transfer_date_to_utc(group['time'].min())
      latest_review_date = transfer_date_to_utc(group['time'].max())

      if gmap_id not in out_dic:
          out_dic[gmap_id] = {
              "gmap_id": gmap_id,
              "reviews": reviews,
              "earliest_review_date": earliest_review_date,
              "latest_review_date": latest_review_date
          }
      else:
          # update the existing entry if the gmap_id already exists in the dictionary
          out_dic[gmap_id]["reviews"].extend(reviews)
          # Update the earliest & latest review dates
          out_dic[gmap_id]["earliest_review_date"] = min(out_dic[gmap_id]["earliest_review_date"], earliest_review_date)
          out_dic[gmap_id]["latest_review_date"] = max(out_dic[gmap_id]["latest_review_date"], latest_review_date)

This function reformats a string containing picture dimensions, then, returns the reformatted string with properly formatted picture dimensions.

In [18]:
def reformat_pic_dim(dim_str):
  pic_dim_match = re.compile(r'\[\s*\[\s*"\d+"\s*,\s*"\d+"\s*\](?:\s*,\s*\[\s*"\d+"\s*,\s*"\d+"\s*\])*\s*\]')
  # Replace the matched pattern, ensuring no extra spaces before sub-lists and formatting them properly
  pic_dim = pic_dim_match.sub(lambda x: x.group(0).replace('\n', '').replace(' ', ''), dim_str)
  return pic_dim

In [19]:
output_json = json.dumps(out_dic, indent=2, ensure_ascii=False)
output_json = reformat_pic_dim(output_json)

In [20]:
with open('/task1_output.json', 'w', encoding='utf-8') as output_file:
  output_file.write(output_json)